---
title: "Snowflake Storage Cortex Forecast Example"
date: "08/25/2024"
format:
    html:
        code-fold: False
        output: False
---

# Overview

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#| hide
from nbdev.showdoc import *

# Example Useage

In [3]:
#| skip
from snowflake.snowpark.version import VERSION
from cortex_forecast.forecast import SnowflakeMLForecast
import os


In [4]:
#| skip
forecast_model = SnowflakeMLForecast(
    config_file='./cortex_forecast/files/yaml/storage_forecast_config.yaml',
    connection_config={
        'user': os.getenv('SNOWFLAKE_USER'),
        'password': os.getenv('SNOWFLAKE_PASSWORD'),
        'account': os.getenv('SNOWFLAKE_ACCOUNT'),
        'database': 'CORTEX',
        'warehouse': 'CORTEX_WH',
        'schema': 'DEV',
        'role': 'CORTEX_USER_ROLE'  # Use the desired role
    }
)

snowflake_environment = forecast_model.session.sql('SELECT current_user(), current_version()').collect()
snowpark_version = VERSION
print('\nConnection Established with the following parameters:')
print('User                        : {}'.format(snowflake_environment[0][0]))
print('Role                        : {}'.format(forecast_model.session.get_current_role()))
print('Database                    : {}'.format(forecast_model.session.get_current_database()))
print('Schema                      : {}'.format(forecast_model.session.get_current_schema()))
print('Warehouse                   : {}'.format(forecast_model.session.get_current_warehouse()))
print('Snowflake version           : {}'.format(snowflake_environment[0][1]))
print('Snowpark for Python version : {}.{}.{}'.format(snowpark_version[0], snowpark_version[1], snowpark_version[2]))



Connection Established with the following parameters:
User                        : JD_SERVICE_ACCOUNT_ADMIN
Role                        : "CORTEX_USER_ROLE"
Database                    : "CORTEX"
Schema                      : "DEV"
Warehouse                   : "CORTEX_WH"
Snowflake version           : 8.31.1
Snowpark for Python version : 1.19.0


In [5]:
#| skip
# Create Training Data
training_days = 60
predicted_days = 30

forecast_model.session.sql(f'''CREATE OR REPLACE TABLE storage_usage_train AS
    SELECT 
        TO_TIMESTAMP_NTZ(usage_date) AS usage_date,
        storage_bytes / POWER(1024, 3) AS storage_gb
    FROM snowflake.account_usage.storage_usage
    WHERE TO_TIMESTAMP_NTZ(usage_date) < DATEADD(day, -{training_days}, CURRENT_DATE())
''').collect()
forecast_model.session.sql('SELECT * FROM storage_usage_train LIMIT 10').show()

--------------------------------------------
|"USAGE_DATE"         |"STORAGE_GB"        |
--------------------------------------------
|2024-04-02 00:00:00  |181.23798023629934  |
|2024-03-08 00:00:00  |174.59878158662468  |
|2024-05-11 00:00:00  |210.28794746939093  |
|2024-05-30 00:00:00  |210.35030308552086  |
|2024-02-22 00:00:00  |7.973375583067536   |
|2024-03-04 00:00:00  |7.973911106586456   |
|2024-02-09 00:00:00  |7.942007430829108   |
|2024-01-26 00:00:00  |5.264182310551405   |
|2024-03-14 00:00:00  |184.33576492313296  |
|2024-06-20 00:00:00  |260.5835631471127   |
--------------------------------------------



In [6]:
# Run Forecast
forecast_data = forecast_model.create_and_run_forecast()

2024-08-20 18:38:35.559 
  command:

    streamlit run /Users/jdemlow/miniconda3/envs/spacy_project/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]


Creating tag: CREATE OR REPLACE TAG environment COMMENT = 'Specifies the production';
Creating tag: CREATE OR REPLACE TAG team COMMENT = 'Specifies the data_science';
Step 1/4: Creating training table...
Step 2/4: Creating forecast model...
Generated SQL:

        CREATE OR REPLACE SNOWFLAKE.ML.FORECAST my_forecast_model_20240820_lszyz(
            INPUT_DATA => SYSTEM$REFERENCE('table', 'storage_usage_train'),
            TIMESTAMP_COLNAME => 'usage_date',
            TARGET_COLNAME => 'storage_gb',
        CONFIG_OBJECT => OBJECT_CONSTRUCT('on_error', 'skip', 'evaluate', TRUE, 'evaluation_config', OBJECT_CONSTRUCT('n_splits', 2, 'gap', 0, 'prediction_interval', 0.95))) WITH TAG (environment = 'production', team = 'data_science') COMMENT = 'Forecast model for predicting sales trends.';
Step 3/4: Generating forecasts...
Configuration Details:
Forecast Days: 30
Output Table: storage_forecast_results
Evaluation Config: {'n_splits': 2, 'gap': 0, 'prediction_interval': 0.95}
Generated Fore

In [7]:
# Generate Forecast and Visualization
forecasting_period = 30
confidence_interval = 0.95
forecast_model.generate_forecast_and_visualization(forecasting_period, confidence_interval)

AttributeError: 'SnowflakeMLForecast' object has no attribute 'generate_forecast_and_visualization'